In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from prefect import Flow, Parameter, unmapped
import pandas as pd
from prefect.engine.executors import DaskExecutor
from ml_flow import data_cleaning_flow
from tinydb import TinyDB

In [4]:
input_df = pd.read_csv("../data/titanic/train.csv")
test= pd.read_csv("../data/titanic/test.csv")

In [5]:
executor = DaskExecutor()
data_cleaner = data_cleaning_flow.run(
    input_data= input_df, 
    problem="binary classification", 
    target = "Survived", 
    features = "infer",
    executor=executor
)

[2020-04-13 05:50:42,269] INFO - prefect.FlowRunner | Beginning Flow run for 'data_cleaning'
[2020-04-13 05:50:42,274] INFO - prefect.FlowRunner | Starting flow run.
[2020-04-13 05:50:42,538] INFO - prefect.TaskRunner | Task 'target': Starting task run...
[2020-04-13 05:50:42,544] INFO - prefect.TaskRunner | Task 'input_data': Starting task run...
[2020-04-13 05:50:42,583] INFO - prefect.TaskRunner | Task 'target': finished task run for task with final state: 'Success'
[2020-04-13 05:50:42,609] INFO - prefect.TaskRunner | Task 'input_data': finished task run for task with final state: 'Success'
[2020-04-13 05:50:42,754] INFO - prefect.TaskRunner | Task 'problem': Starting task run...
[2020-04-13 05:50:42,768] INFO - prefect.TaskRunner | Task 'problem': finished task run for task with final state: 'Success'
[2020-04-13 05:50:42,776] INFO - prefect.TaskRunner | Task 'features': Starting task run...
[2020-04-13 05:50:42,785] INFO - prefect.TaskRunner | Task 'features': finished task run f

[2020-04-13 05:50:43,514] INFO - prefect.TaskRunner | Task 'fit_target_encoder': finished task run for task with final state: 'Success'
[2020-04-13 05:50:43,544] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': Starting task run...
[2020-04-13 05:50:43,545] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': Starting task run...
[2020-04-13 05:50:43,575] INFO - prefect.TaskRunner | Task 'target_encoder_transform': Starting task run...
[2020-04-13 05:50:43,577] INFO - prefect.TaskRunner | Task 'target_encoder_transform': Starting task run...
[2020-04-13 05:50:43,584] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': finished task run for task with final state: 'Success'
[2020-04-13 05:50:43,593] INFO - prefect.TaskRunner | Task 'transform_yeo_johnson_transformer': finished task run for task with final state: 'Success'
[2020-04-13 05:50:43,630] INFO - prefect.TaskRunner | Task 'target_encoder_transform': finished task run for tas

In [6]:
from meta_model import MetaModel, meta_model_flow
executor = DaskExecutor()
meta_model_run = meta_model_flow.run(
    train_data = "transformed_train.df",
    valid_data = "transformed_valid.df",
    train_target = "train_target.df",
    tinydb = "db.json",
    #problem = "binary classification",
    executor = executor,
   
)

ValueError: Flow.run did not receive the following required parameters: problem

In [ ]:
from meta_model import MetaModel, meta_model_flow
from tinydb import TinyDB
tinydb = TinyDB("db.json")
meta_model = MetaModel(problem="regression", db=tinydb,)

In [ ]:
dir(meta_model)

In [ ]:
with Flow("data_visualization") as flow:
    transformed_train_df = pd.read_feather("transformed_train.df")
    transformed_valid_df = pd.read_feather("transformed_valid.df")
    svd = fit_svd(transformed_train_df)
    svd_train = svd_transform(svd, transformed_train_df, "transformed_train_df",tinydb)
    svd_valid = svd_transform(svd, transformed_valid_df, "transformed_valid_df",tinydb)
    

In [ ]:
#flow.visualize(flow_state=flow_state)

In [ ]:
from tinydb import TinyDB, Query
db = TinyDB("db.json")
db.all()

In [ ]:
q = Query()
r = db.search(q.chunk == "svdname")

In [ ]:
dir(q.chunk

In [ ]:
flow._sorted_tasks()[38]

In [ ]:
dir(flow._sorted_tasks()[38])